#### Install the required packages 

In [ ]:
!pip install mysql-connector-python

In [ ]:
!pip install PyMySQL

#### Import the required libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose 
import datetime 
import itertools
plt.style.use("fivethirtyeight")
import statsmodels.api as sm
import matplotlib
import warnings
import math
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine
from project_lib import Project
warnings.filterwarnings('ignore')
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12

#### Fill the following details 

In [ ]:
host = ""
username = ""
password = ""
database = "iotprediction"

In [ ]:
credentials = {"host": host, 
               "username": username, 
               "password": password, 
               "database": database}

conn = mysql.connector.connect( user=credentials['username'], password=credentials['password'], host=credentials['host'], database=credentials['database'])

cursor = conn.cursor()

cursor.execute("SELECT * FROM data")

myresult = cursor.fetchall()

field_names = [i[0] for i in cursor.description]

df_data_1= pd.DataFrame(myresult, columns = field_names)
df = df_data_1.sort_values(by=['id'])
df.columns = ['id', 'date', 'temp']
df = df.dropna()
df

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M')
df['full-date'] = [d.date() for d in df['date']]
df['year'] = df['date'].apply(lambda x : x.year)
df['month'] = df['date'].apply(lambda x : x.month)
df['day'] = df['date'].apply(lambda x : x.day)
df.head()

#### Time series analysis

In [ ]:
grouped_single = df.groupby(['year','month','day','full-date']).agg({'temp': ['mean']})
grouped_single.columns = ['temp_mean']
grouped_single = grouped_single.reset_index()
grouped_single = grouped_single.set_index('full-date')
y = grouped_single.temp_mean
print(y)
print("")
print("Average temperature plot daywise")
y.plot(figsize = (20,6))
plt.show()

In [ ]:
p = d = q = range(0,2)
pdq = list(itertools.product(p,d,q))
seasonal_pdq = [(x[0],x[1],x[2],1) for x in list(itertools.product(p,d,q))]

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order = param,
                                            seasonal_order = param_seasonal,
                                            enforce_stationarity = False,
                                            enforce_invertibility = False)
            result = mod.fit()
        except :
            continue

mod = sm.tsa.statespace.SARIMAX(y,
                               order =(1,1,1),
                               param_seasonal = (0,0,0,1),
                               enforce_stationarity = False,
                               enforce_invertibility = False)
result = mod.fit()

dates_list=[]
for k,v in y.items():
    dates_list.append(str(k)) 
date_index = int((len(dates_list))/2)
startdate = datetime.datetime.strptime(dates_list[0], "%Y-%m-%d").date()
enddate = datetime.datetime.strptime(dates_list[date_index-1], "%Y-%m-%d").date()
predictdate = datetime.datetime.strptime(dates_list[date_index], "%Y-%m-%d").date()

In [ ]:
pred = result.get_prediction(start = enddate,dynamic = False)
pred_ci = pred.conf_int()
ax = y[startdate:enddate].plot(label = 'observed')

In [ ]:
pred = result.get_prediction(start = pd.to_datetime(startdate),dynamic = False)
pred_ci = pred.conf_int()
ax = y[startdate:].plot(label = 'observed')

In [ ]:
print("Predicted and actual temperature table")
y_Forecasted = pred.predicted_mean
y_truth = y[predictdate:]
print(y_truth)

In [ ]:
actual_pred_table = y_truth.to_frame().join(y_Forecasted.to_frame())
actual_pred_table.columns =['temp_mean','predicted_mean']
actual_pred_table['error'] = actual_pred_table.apply(lambda x : x.predicted_mean - x.temp_mean,axis = 1)
actual_pred_table['error_square'] = actual_pred_table.apply(lambda x : x.error**2,axis = 1 )
print(actual_pred_table)
print("")
print("Error difference between actaul and predicted temperature is ", actual_pred_table.error.mean())
print("mean square error difference between actaul and predicted temperature is ", round(math.sqrt(actual_pred_table.error_square.mean()),0))

#### Prediction of temperature for the next 6 days

In [ ]:
pred_uc = result.get_forecast(steps=6)
pred_ci = pred_uc.conf_int()
print("prediction of average temperature for next 6 days is")
pred_ci.index.name = 'foo'
pred_ci.columns =['min','max'] 
pred_ci['Date'] = pred_ci.index
pred_ci.reset_index(drop=True, inplace=True)
pred_ci

#### Visual representation of the prediction

In [ ]:
pred_ci.plot('Date',figsize = (20,6))
plt.axhline(y=31, color='r', linestyle='--')
plt.axhline(y=27, color='r', linestyle='--')
plt.show()

#### Alerts from the prediction  

In [ ]:
print("On following dates the temperature could fall below minimum threshold")
for index, row in pred_ci.loc[(pred_ci['min'] < 27)].iterrows():
    print(pd.to_datetime(row['Date']).date())
    
print("On following dates the temperature could raise above maximum threshold")
for index, row in pred_ci.loc[(pred_ci['max'] > 31)].iterrows():
    print(pd.to_datetime(row['Date']).date())

#### Store the data back in SingleStore Database for visualisation

In [ ]:
# Insert the dataframe into singlestore
cursor.execute("DROP TABLE IF EXISTS predictions")
engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'. 
                                               format(credentials['username'], 
                                                      credentials['password'], 
                                                      credentials['host'], 
                                                      credentials['database']))
pred_ci.to_sql('predictions', con = engine, if_exists = 'append', chunksize = 1000, index= False)

conn = mysql.connector.connect( user=credentials['username'], password=credentials['password'], host=credentials['host'], database=credentials['database'])

cursor = conn.cursor()

cursor.execute("SELECT * FROM predictions")

myresult = cursor.fetchall()

field_names = [i[0] for i in cursor.description]

df_data_1= pd.DataFrame(myresult, columns = field_names)
df_data_1

In [ ]:
conn.close()

In [ ]:
# Save the file locally
project = Project.access()
project.save_data("temp-predictions.csv", df_data_1.to_csv(index=False), overwrite = True)